In [45]:
import pickle

#  Cargar el modelo guardado desde el escritorio
ruta_local = r"C:\Users\nacho\Desktop\modelo_prophet_granularidades.pkl"

with open(ruta_local, "rb") as archivo:
    modelo_cargado = pickle.load(archivo)

#  Verificar qué contiene el archivo
print("Claves en el archivo:", modelo_cargado.keys())  # Debe mostrar ['modelos_granulares', 'predicciones_granulares']

#  Ver las granularidades disponibles
print("Granularidades disponibles:", list(modelo_cargado["modelos_granulares"].keys()))

#  Elegir una granularidad específica para revisar
granularidad = "Día a día"  # Cambia esto si quieres ver otra granularidad
modelo_seleccionado = modelo_cargado["modelos_granulares"][granularidad]
predicciones_seleccionadas = modelo_cargado["predicciones_granulares"][granularidad]


print(predicciones_seleccionadas.head())

Claves en el archivo: dict_keys(['modelos_granulares', 'predicciones_granulares'])
Granularidades disponibles: ['Día a día', 'Semana a semana', 'Mes a mes', 'Trimestre a trimestre', 'Semestre a semestre', 'Año a año']
          ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0 2011-01-01  0.605442    0.134140    0.300886     0.605442     0.605442   
1 2011-01-02  0.605342    0.284579    0.457297     0.605342     0.605342   
2 2011-01-03  0.605242    0.551229    0.719795     0.605242     0.605242   
3 2011-01-04  0.605141    0.603793    0.783358     0.605141     0.605141   
4 2011-01-05  0.605041    0.619021    0.792645     0.605041     0.605041   

   additive_terms  additive_terms_lower  additive_terms_upper  es_festivo  \
0       -0.393138             -0.393138             -0.393138   -0.270674   
1       -0.234120             -0.234120             -0.234120    0.000000   
2        0.029267              0.029267              0.029267    0.000000   
3        0.086156

In [46]:
import pickle
import pandas as pd
import plotly.graph_objects as go

#  Ruta del archivo modelo en local
ruta_local = r"C:\Users\nacho\Desktop\modelo_prophet_granularidades.pkl"

#  Cargar el modelo guardado en local
with open(ruta_local, "rb") as archivo:
    modelo_cargado = pickle.load(archivo)

#  Ruta del archivo de datos reales
ruta_datos = r"C:\Users\nacho\Desktop\Aprendiendo a programar\PFB\TFB\data\train_test\train_temporal.csv"

#  Cargar datos reales de entrenamiento
df_entrenamiento = pd.read_csv(ruta_datos)

#  Convertir fechas a datetime
df_entrenamiento["ds"] = pd.to_datetime(df_entrenamiento["fecha"])
df_entrenamiento = df_entrenamiento.rename(columns={"valor_(GWh)": "y"})

#  **Restaurar todos los datos históricos**
df_entrenamiento = df_entrenamiento.copy()  # 🔹 Mantiene todo el historial (2011-2022)

#  Definir granularidades
granularidades = ["Día a día", "Semana a semana", "Mes a mes", "Trimestre a trimestre"]

#  Diccionario de colores para las predicciones
colores = {"Día a día": "blue", "Semana a semana": "red", "Mes a mes": "green", "Trimestre a trimestre": "orange"}

#  Iterar sobre granularidades y graficar cada una correctamente
for granularidad in granularidades:
    #  Extraer predicciones de la granularidad
    predicciones = modelo_cargado["predicciones_granulares"][granularidad]

    #  Generar df_futuro sin recortar fechas manualmente
    df_futuro = modelo_cargado["modelos_granulares"][granularidad].make_future_dataframe(
        periods=len(predicciones), freq="D"
    )

    #  Crear figura para cada granularidad
    fig = go.Figure()

    #  Datos históricos completos
    fig.add_trace(go.Scatter(
        x=df_entrenamiento["ds"], y=df_entrenamiento["y"],
        mode="markers", name="Datos Históricos",
        marker=dict(color="black", opacity=0.5)
    ))

    #  Predicción de la granularidad
    fig.add_trace(go.Scatter(
        x=predicciones["ds"], y=predicciones["yhat"],
        mode="lines", name=f"Predicción - {granularidad}",
        line=dict(color=colores[granularidad], width=2)
    ))

    # Configurar diseño de la gráfica
    fig.update_layout(
        title=f"Predicción con granularidad: {granularidad}",
        xaxis_title="Fecha",
        yaxis_title="Demanda en GWh",
        template="plotly_white",
        hovermode="x unified"
    )

    # Mostrar la gráfica de la granularidad actual
    fig.show()



In [47]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error


ruta_local = r"C:\Users\nacho\Desktop\modelo_prophet_granularidades.pkl"

#  Cargar el modelo guardado en local
with open(ruta_local, "rb") as archivo:
    modelo_cargado = pickle.load(archivo)

#  Ruta del archivo de datos reales
ruta_datos = r"C:\Users\nacho\Desktop\Aprendiendo a programar\PFB\TFB\data\train_test\train_temporal.csv"

#  Cargar datos reales de entrenamiento
df_entrenamiento = pd.read_csv(ruta_datos)

#  Convertir fechas a datetime
df_entrenamiento["ds"] = pd.to_datetime(df_entrenamiento["fecha"])
df_entrenamiento = df_entrenamiento.rename(columns={"valor_(GWh)": "y"})

# **Filtrar solo datos desde 2020 en adelante (si quieres evaluar solo datos recientes)**
df_entrenamiento = df_entrenamiento[df_entrenamiento["ds"] >= "2020-01-01"]

#  Seleccionar la granularidad principal a evaluar
granularidad = "Día a día"  # Puedes cambiar a otra granularidad

#  Extraer predicciones de la granularidad seleccionada
predicciones = modelo_cargado["predicciones_granulares"][granularidad]

#  Unir datos reales y predicciones en un mismo DataFrame
df_evaluacion = df_entrenamiento.merge(predicciones, on="ds", how="inner")

mse = mean_squared_error(df_evaluacion["y"], df_evaluacion["yhat"])
mae = mean_absolute_error(df_evaluacion["y"], df_evaluacion["yhat"])
rmse = np.sqrt(mse)  # Raíz cuadrada del error cuadrático medio

correlacion = df_evaluacion[["y", "yhat"]].corr().iloc[0, 1]

resultados = {
    "MSE (Error Cuadrático Medio)": mse,
    "RMSE (Raíz del Error Cuadrático Medio)": rmse,
    "MAE (Error Absoluto Medio)": mae,
    "Correlación Datos Reales vs Predicción": correlacion
}

for clave, valor in resultados.items():
    print(f"{clave}: {valor:.6f}")


MSE (Error Cuadrático Medio): 0.006049
RMSE (Raíz del Error Cuadrático Medio): 0.077773
MAE (Error Absoluto Medio): 0.056725
Correlación Datos Reales vs Predicción: 0.879550
